### GCCをWindowsへインストールするぞ
---

PyStanの公式にはAnacondaを使えって書いてあるけど、純粋に使いたくない

#### PyStan の解説ページ
---

https://pystan.readthedocs.io/en/latest/windows.html#windows

#### 1. windowsへ MinGW-w64 をインストール
---

参考サイト  
https://qiita.com/yut-nagase/items/9bb65481685f6b8002de  
  
インストール先の.batファイルを参考にして、Pathを通しておく  
  
上記サイトとのオプション設定の違いは下記の通り  

* Architecture: x86_64
* Exception: seh

  
最終的に↓をパスに通す感じになるはず  
C:\Program Files\mingw-w64\x86_64-8.1.0-posix-seh-rt_v6-rev0\mingw64\bin

#### 2. distutils で 使用するコンパイラを設定
---

参考サイト  
https://qiita.com/nocd5/items/a6db920dc86a41b6625d  
  
distutils の場所の調べ方  
venvでやるときは、作った仮想環境のパスではなく、  
参照元にしているオリジナルのPython置き場になる

```
>>> import distutils
>>> print(distutils . __file__)
```

PythonをコンパイルしたMSVCのバージョン次第で、distutilsが機能しないので、  
~/Lib/ditutils/cygwinccompiler.py を手動パッチ当てする  
  
参考サイト  
https://stackoverflow.com/questions/34135280/valueerror-unknown-ms-compiler-version-1900/34427014

#### なんかそのままだと引数が足りないらしい
---

理由もよく分かってないけど、デフォルトのままだと何かの参照に失敗するらしい  
PyStanではなく、MinGwを使ったコンパイル自体に失敗しているっぽい
  
mingwでのリンクエラーについて  
https://teratail.com/questions/83914

#### gcc.exe の実行に対して、-DMS_WIN64 を引数として追加
---

↑のサンプルコードを動かしながらログを見るなどしてみた結果、  
gcc.exe の実行へ -DMS_WIN64 を追加するとなんとかなるっぽい
  
cpython to pyd binary - "long" converted to "int" badly before shift / division operator  
https://github.com/cython/cython/issues/2670#issuecomment-432212671

#### デフォルト設定にする方法がわからん
---

とりあえず、pystanなら、↓のように呼べばなんとかなるけど...  

```sm = pystan.StanModel(model_code=model, verbose=False, extra_compile_args=["-DMS_WIN64"])```

#### デフォルト設定にしたければ、distutils を手動で修正しろって書いてある
---

https://groups.google.com/forum/#!topic/cython-users/LBeFrAypd2E

```
Find_And_replace in "..\Lib\distutils\cygwinccompiler.py"
this
"-O -W"
per that
"-O -DMS_WIN64 -W"
```

#### 32bit じゃないけどいいか一抹の心配
---

設定ファイルにも ```compiler=mingw32``` とか書きこんでるし、  
実際は64bit版使ってるけど大丈夫かわずかに気になる  
公式にはMinGW 64bit でインスコしろって書いてあるからいいか